In [ ]:
import pandas as pd
import datetime as dt

#Блок продуктовая аналитика

In [ ]:
data = pd.read_csv('Данные для вступительных задач. Kolesa Academy DA 2024 - Данные об аудитории.csv')

In [ ]:
data

,date,user_id,view_adverts
0,2023-11-11,8c020470-8461-11ed-83d0-552e8cc749d6,13
1,2023-11-18,5875f070-7b92-11ee-a6fb-8b298e83f4f7,14
2,2023-11-29,3c2d27c0-4fd6-11eb-b89f-2ffb31b67dd6,21
3,2023-11-29,234a96d0-ad16-11ed-a2e6-793ddfeeba1f,23
4,2023-11-29,4d07c180-644f-11eb-879c-b7c02edf4f37,12
...,...,...,...
16809,2023-11-17,39df47d0-67e5-11ee-8fb4-6fca0b38dcdb,9
16810,2023-11-05,211ecf40-52f0-11ee-8000-aff5e9624248,9
16811,2023-11-16,ae274910-82d1-11ee-b2b8-1d1441fa3755,9
16812,2023-11-13,7b3d2810-e585-11ed-a524-d983167b6653,9


Во вкладке "Данные об аудитории" информация о пользователях, посетивших наше приложение в ноябре. Чему равен MAU продукта?

In [ ]:
data['date'] = pd.to_datetime(data['date'])

In [ ]:
mau = data.groupby(data['date'].dt.to_period('M'))['user_id'].nunique()

In [ ]:
mau

date
2023-11    7639
Freq: M, Name: user_id, dtype: int64

In [ ]:
data['date'] = pd.to_datetime(data['date'])

Используя вкладку "Данные об аудитории", посчитайте, чему будет равен DAU

In [ ]:
# Вычисляем DAU
dau = data.groupby('date')['user_id'].nunique()

In [ ]:
average_dau = dau.mean()

In [ ]:
average_dau

560.4666666666667

Посчитайте, чему будет равен retention первого дня у пользователей, пришедших в продукт 1 ноября

In [ ]:
# Выберем только пользователей, которые пришли в продукт 1 ноября
users_nov_1 = data[data['date'] == '2023-11-01']['user_id'].unique()

# Выберем пользователей, которые пришли 1 ноября и вернулись на следующий день
next_day = data[data['date'] == '2023-11-02']['user_id'].unique()
retention_1st_day = (len(set(next_day) & set(users_nov_1)) / len(users_nov_1)) * 100

In [ ]:
retention_1st_day

26.64526484751204

Есть информация о том, сколько объявлений посмотрел каждый пользователь (view_adverts). Посчитайте пользовательскую конверсию в просмотр объявления за ноябрь? (в пользователях)

In [ ]:
# Выберем только просмотры объявлений за ноябрь
november_views = data[data['date'].dt.month == 11]

# Подсчитаем уникальных пользователей, просмотревших хотя бы одно объявление в ноябре
unique_users_november = november_views['user_id'].nunique()

# Посчитаем пользовательскую конверсию в просмотр объявления за ноябрь
conversion_rate_november = (unique_users_november / data['user_id'].nunique()) * 100

In [ ]:
conversion_rate_november

100.0

Посчитайте среднее количество просмотренных объявлений на пользователя в ноябре

In [ ]:
# Выбираем только записи за ноябрь
november_data = data[data['date'].dt.month == 11]

# Посчитаем общее количество просмотренных объявлений
total_views = november_data['view_adverts'].sum()

# Посчитаем количество уникальных пользователей, просмотревших объявления
unique_users = november_data['user_id'].nunique()

# Рассчитываем среднее количество просмотренных объявлений на пользователя
average_views_per_user = total_views / unique_users

In [ ]:
average_views_per_user

2.8687000916350307

#Блок A/B тестирования

Kolesa - площадка для продажи авто. Продавцы находят больше потенциальных покупателей, купив платные услуги (ПУ) продвижения своего объявления. После применения этих платных услуг объявления находятся в топе и покупатели чаще их видят.

В команде монетизации часто вносят изменения в механики работы, интерфейс ПУ. Каждое такое изменение проходит через АБ тест. Наиболее часто мы следим за метрикой ARPU = общая выручка/общее количество пользователей.

Во вкладке "Данные АБ-тестов" результаты трех несвязанных АБ тестов для этой метрики.

**Посмотрите на результаты тестов и проинтерпретируйте их. Напишите значения p-value, которые вы получили.
Подготовьте выводы и рекомендации. Какую механику стоит масштабировать по-вашему?**

* experiment_num - номер эксперимента

* experiment_group - группа, в которую попал пользователь

* user_id - id пользователя

* revenue - выручка, которую сгенерировал пользователь, купив платную услугу продвижения

In [ ]:
data2 = pd.read_csv('Данные для вступительных задач. Kolesa Academy DA 2024 - Данные АБ тестов.csv')

In [ ]:
data2

,experiment_num,experiment_group,user_id,revenue
0,1,test,38456,520
1,1,control,13125924,806
2,1,control,9761984,0
3,1,test,11387012,208
4,1,test,18319648,104
...,...,...,...,...
2830,3,test,725230,0
2831,3,test,15219649,0
2832,3,test,18648281,0
2833,3,test,244892,0


In [ ]:
from scipy.stats import ttest_ind

In [ ]:
# Разделение данных по экспериментам
experiment_1 = data2[data2['experiment_num'] == 1]
experiment_2 = data2[data2['experiment_num'] == 2]
experiment_3 = data2[data2['experiment_num'] == 3]

In [ ]:
# Проведение статистического анализа для каждого эксперимента
for experiment in [experiment_1, experiment_2, experiment_3]:
    group_test = experiment[experiment['experiment_group'] == 'test']['revenue']
    group_control = experiment[experiment['experiment_group'] == 'control']['revenue']
    t_statistic, p_value = ttest_ind(group_test, group_control)
    print("Experiment:", experiment['experiment_num'].iloc[0])
    print("T-Statistic:", t_statistic)
    print("P-Value:", p_value)
    if p_value < 0.05:
        print("Статистически значимые различия")
    else:
        print("Нет статистически значимых различий")
    print()

Experiment: 1
T-Statistic: -0.4006287660577395
P-Value: 0.688784211779017
Нет статистически значимых различий

Experiment: 2
T-Statistic: -3.303268516112734
P-Value: 0.0009915972237576193
Статистически значимые различия

Experiment: 3
T-Statistic: 1.8703638838987622
P-Value: 0.06174290011218765
Нет статистически значимых различий



Выводы:

- Эксперимент 2 демонстрирует статистически значимые различия между тестовой и контрольной группами. Это говорит о том, что механика, протестированная в эксперименте 2, имеет потенциал для масштабирования и дальнейшего внедрения. Из 2 эксперимента следует, что рекомендуется более детально изучить и масштабировать эту механику.

- Эксперименты 1 и 3 не показали статистически значимых различий между тестовой и контрольной группами, что может указывать на неэффективность протестированных механик. Для экспериментов 1 и 3 можно провести дополнительный анализ.

#Блок математика

In [ ]:
df = pd.read_csv('Данные для вступительных задач. Kolesa Academy DA 2024 - Листеры.csv')

In [ ]:
df

,user_id,date,cnt_adverts,age,cnt_contacts,revenue
0,100,2022-01-01,6,21,119,53
1,100,2022-01-02,2,21,200,18
2,100,2022-01-03,6,21,193,42
3,100,2022-01-04,2,21,143,38
4,100,2022-01-05,2,21,190,40
...,...,...,...,...,...,...
153,130,2022-01-27,1,22,185,60
154,130,2022-01-19,0,22,86,41
155,130,2022-01-28,0,22,183,5
156,130,2022-01-30,0,22,86,12


По датасету с листерами посчитайте средний доход на пользователя

In [ ]:
df['revenue'].mean()

30.70253164556962

По датасету с листерами посчитайте медиану возраста пользователя

In [ ]:
df['age'].median()

28.0

По данным с листетрами рассчитайте коэффициент линейной корреляции Пирсона между следующими переменными age и cnt_contacts - чтобы понять, есть ли зависимость между возрастом пользователя и количеством контактов

In [ ]:
correlation = df['age'].corr(df['cnt_contacts'])

In [ ]:
correlation

-0.0701029547333032